In [1]:
import requests
from bs4 import BeautifulSoup
import re
from copy import deepcopy
import csv
import itertools

bigrams = lambda l: list(zip(l[:-1], l[1:]))
new_sent_sign = '<NEWSENT>'

In [2]:
page = requests.get('https://math.nist.gov/quantum/zoo/')
contents = str(page.content)
contents = contents.replace('<b>Algorithm: </b>', '<b>Algorithm:</b>')
contents = contents.replace('<b>Description: </b>', '<b>Description:</b>')
# soup = BeautifulSoup(contents, 'html.parser') #don't even need bs, as the page is very non-semantic anyway

In [3]:
indices = bigrams([m.start() for m in re.finditer('<b>Algorithm:</b>', contents)]+[len(contents)])
complete_txts = [str(contents)[i[0]:i[1]] for i in indices]

In [4]:
all_algos = []
for i in complete_txts:
    this_algorithm = {}
    for name, searchstring in [['name', '<b>Algorithm:</b>'], ['speedup_coarse', '<b>Speedup:</b>'], ['description', '<b>Description:</b>']]:
        tmp = i[i.find(searchstring)+len(searchstring):]
        this_algorithm[name] = tmp[:tmp.find('<br />')].strip()
        
    
    all_algos.append(this_algorithm)

In [5]:
def replace_dots(txt):
    positions = [m.start() for m in re.finditer('\.', tmp)]
    positions_new = []
    for i in positions:
        try:
            first_letter = tmp[i+1:].strip()[0]
            if first_letter.isupper():
                positions_new.append(i)
        except IndexError:
            pass
    for i in sorted(positions_new, reverse=True):
        txt = txt[:i]+new_sent_sign+txt[i+1:]
    return txt

for algo_num in range(len(all_algos)):
    tmp = all_algos[algo_num]['description'].replace('\\n', ' ').replace('  ', ' ')
    tmp = re.compile(r'<.*?>').sub('', tmp)
    tmp = re.compile(r'\[.*?\]').sub('', tmp)
    for i in range(1000):
        tmp = tmp.replace('. ', '.').replace(' .','.')
    tmp = tmp.replace('whereas', new_sent_sign)
    
    tmp = replace_dots(tmp)
    
    for i in range(1000, 1, -1):
        tmp = tmp.replace(new_sent_sign*i,'.')
    sents = [i.strip() for i in tmp.split(new_sent_sign)]

    sents_with_formula = []
    for sent in sents:
        if '\\\\(' in sent and '\\\\)' in sent:
            sents_with_formula.append((sent, sent[sent.find('\\\\(')+3:sent.find('\\\\)')]))
           
    #if all_algos[algo_num]['name'] == 'Verifying Matrix Products':
    #    for i in sents:
    #        print(i+'\n\n\n')
     
            
    for sent, formula in sents_with_formula:
        sent = sent.lower()
        if '(' in formula and ('o' in formula.lower() or 'theta' in formula.lower()) and ')' in formula:
            if 'quantum algorithm' in sent or 'quantum computer' in sent:
                #print("\n"+sent+"\n")
                all_algos[algo_num]['assumed_quantum_runtime'] = formula
            elif 'classical algorithm' in sent or 'classical computer' in sent or 'classically' in sent:
                #print("\n"+sent+"\n")
                all_algos[algo_num]['assumed_classical_runtime'] = formula
            if 'quantum query complexity' in sent and not 'assumed_quantum_runtime' in all_algos[algo_num]:
                all_algos[algo_num]['assumed_quantum_runtime'] = formula

In [6]:
for i in all_algos:
    if i['name'] == 'Subset-sum':
        i['assumed_quantum_runtime'] = ' 2^{0.241n} '
        i['assumed_classical_runtime'] = ' 2^{0.291n} '
    if i['name'] == 'Gradients, Structured Search, and Learning Polynomials':
        i['assumed_quantum_runtime'] = 'O(d)'
    if i['name'] == 'Hidden Shift':
        i['assumed_quantum_runtime'] = 'O(log n)'
        i['assumed_classical_runtime'] = ' \Omega(2^{n/2}) '
    if i['name'] == 'Ordered Search':
        i['assumed_quantum_runtime'] = '0.433 \log_2 N'
        i['assumed_classical_runtime'] = ' \log_2 N '
    if i['name'] == 'Graph Properties in the Adjacency Matrix Model' or i['name'] == 'Unit Group':
        try:
            del i['assumed_quantum_runtime']
        except:
            pass
    if i['name'] == 'Junta Testing and Group Testing':
        i['assumed_quantum_runtime'] = ' \widetilde{O}(n \sqrt{k/\epsilon}) '

In [7]:
MATH_MODULE = __import__('math')
ALLOWED_FUNCTIONS = {**{name: getattr(MATH_MODULE, name) for name in dir(MATH_MODULE) if name[0] != '_'}, **{'abs': abs}}
names = [i[0] for i in ALLOWED_FUNCTIONS.items()]
def add_parantheses(a):
    for name in names:
        if a.find(name) >= 0:
            pos = a.find(name)+len(name)
            stuff_after = a[pos:].strip()
            if not stuff_after.startswith("("):
                num_opens = 0
                for num, sign in enumerate(stuff_after):
                    if sign == "(":
                        num_opens += 1
                    elif sign == ")":
                        num_opens -= 1
                    if num_opens < 0:
                        a = a[:pos]+"("+stuff_after[:num]+")"+a[pos+num+1:]
    return a

In [8]:
for i in range(len(all_algos)):
    for string in 'assumed_quantum_runtime', 'assumed_classical_runtime':
        if string in all_algos[i]:
            all_algos[i][string] = all_algos[i][string].replace("\\\\", " \\\\").replace("  "," ")
            all_algos[i][string] = all_algos[i][string].strip().\
                replace('\\\\widetilde{O}', 'O').replace('{','(').replace('}',')').replace('^','**').replace(' (','(').\
                replace('\\\\Omega', 'O').replace('\\Omega', 'O').replace('\\log_2', 'log2').\
                replace('\\\\widetilde(\\\\Theta)','O').replace('\\\\left(','(').replace('\\\\right)',')').\
                replace('( ','(').replace(' )',')')\
                .replace('\\\\','')

            replace_letters = ['q','G','d','k']
            if not re.findall(r"\bn\b", all_algos[i][string]):
                for letter in replace_letters:
                    all_algos[i][string] = re.sub(r"\b%s\b" % letter , 'n', all_algos[i][string])
            all_algos[i][string] = re.sub(r'vert(.*?)vert', 'abs(\g<1>)', all_algos[i][string])
            all_algos[i][string] = re.sub(r'\|(.*?)\|', 'abs(\g<1>)', all_algos[i][string])
            all_algos[i][string] = re.sub(r"log (.)\b", r"log(\g<1>)", all_algos[i][string])
            all_algos[i][string] = re.sub(r"log2 (.)\b", r"log2(\g<1>)", all_algos[i][string])
            all_algos[i][string] = add_parantheses(all_algos[i][string])
    

In [9]:
for i in all_algos:
    if i['name'] == 'Factoring':
        i['meaningful_domain'] = [100000, 150000]
    if i['name'] == 'Subset-sum':
        i['meaningful_domain'] = [1, 70]
    if i['name'] == 'Counterfeit Coins':
        i['meaningful_domain'] = [1, 7000]
    if i['name'] == 'Search with Wildcards':
        i['meaningful_domain'] = [1, 200]

In [10]:
for i in all_algos:
    try:
        i['meaningful_domain_min'] = i['meaningful_domain'][0]
        i['meaningful_domain_max'] = i['meaningful_domain'][1]    
    except:
        pass

In [11]:
all_algos

[{'name': 'Factoring',
  'speedup_coarse': 'Superpolynomial',
  'description': 'Given an <i>n</i>-bit integer, find the prime\\nfactorization. The quantum algorithm of Peter Shor solves this in\\n\\\\( \\\\widetilde{O} (n^3) \\\\) time\\n[<a href="#Shor_factoring">82</a>,<a href="#Shor_factoring94">125</a>]. \\nThe fastest known classical algorithm for integer factorization is the\\ngeneral number field sieve, which is believed to run in time \\\\(\\n2^{\\\\widetilde{O}(n^{1/3})} \\\\). The best rigorously proven upper bound\\non the classical complexity of factoring is \\\\( O(2^{n/4+o(1)}) \\\\) via the Pollard-Strassen algorithm \\n[<a href="#Pol">252</a>, <a href="#Stras">362</a>]. Shor\\\'s\\nfactoring algorithm breaks RSA public-key encryption and the\\nclosely related quantum algorithms for discrete logarithms break the DSA\\nand ECDSA digital signature schemes and the Diffie-Hellman\\nkey-exchange protocol. A quantum algorithm even faster than Shor\\\'s for\\nthe special case o

In [12]:
keys = list(set(itertools.chain(*[list(i.keys()) for i in all_algos])))

with open('algorithms.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_algos)